In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-


Xanadu PennyLane框架 - 基础练习解答

本文件包含对PennyLane基础练习的参考解答。
如果您还没有尝试完成练习，建议先自行尝试再查看解答。

In [ ]:
import pennylane as qml
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
print("===== PennyLane基础练习解答 =====")


练习1: 创建量子设备和简单电路

In [ ]:
print("\n练习1: 创建量子设备和简单电路 - 解答")

def exercise1_solution():
    # 任务1: 创建设备
    dev = qml.device('default.qubit', wires=2)
    
    # 任务2: 定义量子函数
    @qml.qnode(dev)
    def my_circuit():
        qml.Hadamard(wires=0)
        qml.PauliX(wires=1)
        return [qml.expval(qml.PauliZ(0)), qml.expval(qml.PauliZ(1))]
    
    # 任务3&4: 执行电路并打印结果
    result = my_circuit()
    print(f"量子比特0的PauliZ期望值: {result[0]:.6f}")
    print(f"量子比特1的PauliZ期望值: {result[1]:.6f}")
    print("\n电路图:")
    print(qml.draw(my_circuit)())
    
    return result

results1 = exercise1_solution()

理论解释


In [ ]:
print("\n解释:")
print("量子比特0应用Hadamard门后处于(|0⟩+|1⟩)/√2状态，因此<Z0> = 0")
print("量子比特1应用PauliX门后处于|1⟩状态，因此<Z1> = -1")


练习2: 量子态制备和测量

In [ ]:
print("\n练习2: 量子态制备和测量 - 解答")

def exercise2_solution():
    # 任务1: 创建设备
    dev = qml.device('default.qubit', wires=3)
    
    # 任务2: 编写准备GHZ态的量子函数
    @qml.qnode(dev)
    def ghz_state():
        qml.Hadamard(wires=0)
        qml.CNOT(wires=[0, 1])
        qml.CNOT(wires=[0, 2])
        
        return [
            qml.expval(qml.PauliZ(0)),   # <Z0>
            qml.expval(qml.PauliZ(1)),   # <Z1>
            qml.expval(qml.PauliZ(2)),   # <Z2>
            qml.expval(qml.PauliX(0)),   # <X0>
            qml.expval(qml.PauliX(1)),   # <X1>
            qml.expval(qml.PauliX(2)),   # <X2>
            qml.expval(qml.PauliZ(0) @ qml.PauliZ(1) @ qml.PauliZ(2))  # <Z0⊗Z1⊗Z2>
        ]
    
    results = ghz_state()
    
    print(f"<Z0> = {results[0]:.6f}")
    print(f"<Z1> = {results[1]:.6f}")
    print(f"<Z2> = {results[2]:.6f}")
    print(f"<X0> = {results[3]:.6f}")
    print(f"<X1> = {results[4]:.6f}")
    print(f"<X2> = {results[5]:.6f}")
    print(f"<Z0⊗Z1⊗Z2> = {results[6]:.6f}")
    
    print("\n电路图:")
    print(qml.draw(ghz_state)())
    
    return results

results2 = exercise2_solution()

理论解释


In [ ]:
print("\n解释:")
print("对于GHZ态 (|000⟩ + |111⟩)/√2:")
print("1. 每个量子比特单独测量PauliZ的期望值为0，因为它们有50%概率处于|0⟩和50%概率处于|1⟩")
print("2. 每个量子比特单独测量PauliX的期望值接近0，这反映了量子相干性")
print("3. 三个量子比特的PauliZ张量积期望值为1，表明它们是完全相关的 - 要么全部测量为0，要么全部测量为1")


练习3: 参数化电路与自动微分

In [ ]:
print("\n练习3: 参数化电路与自动微分 - 解答")

def exercise3_solution():
    # 任务1: 创建设备
    dev = qml.device('default.qubit', wires=1)
    
    # 任务2: 创建参数化量子电路
    @qml.qnode(dev)
    def param_circuit(params):
        qml.RX(params[0], wires=0)
        qml.RY(params[1], wires=0)
        qml.RZ(params[2], wires=0)
        return qml.expval(qml.PauliZ(0))
    
    # 任务3&4: 对不同参数值执行电路并计算梯度
    x_values = np.linspace(0, 2*np.pi, 50)
    y_values = []
    gradient_values = []
    
    for x in x_values:
        params = np.array([x, np.pi/4, np.pi/3])
        y_values.append(param_circuit(params))
        gradient_values.append(qml.grad(param_circuit)(params)[0])
    
    # 任务5: 绘制电路输出相对于第一个参数的变化图
    plt.figure(figsize=(10, 6))
    plt.plot(x_values, y_values, 'b-', label='电路输出')
    plt.plot(x_values, gradient_values, 'r--', label='梯度')
    plt.xlabel('RX角度参数')
    plt.ylabel('PauliZ期望值')
    plt.title('参数化量子电路的输出和梯度')
    plt.grid(True)
    plt.legend()
    plt.savefig('param_circuit_output.png')
    plt.close()
    
    print("创建的参数化电路:")
    print(qml.draw(param_circuit)(np.array([np.pi/4, np.pi/3, np.pi/2])))
    print("\n对于参数 [π/4, π/3, π/2]:")
    params = np.array([np.pi/4, np.pi/3, np.pi/2])
    output = param_circuit(params)
    gradient = qml.grad(param_circuit)(params)
    print(f"电路输出: {output:.6f}")
    print(f"梯度: {gradient}")
    print("\n已绘制电路输出和梯度随第一个参数变化的图表，保存为'param_circuit_output.png'")
    
    return y_values, gradient_values

y_values, gradient_values = exercise3_solution()


练习4: 使用PennyLane模板

In [ ]:
print("\n练习4: 使用PennyLane模板 - 解答")

def exercise4_solution():
    # 任务1: 创建设备
    dev = qml.device('default.qubit', wires=4)
    
    # 任务2-4: 使用模板创建电路
    @qml.qnode(dev)
    def template_circuit(features, weights):
        # 使用AngleEmbedding嵌入特征
        qml.templates.AngleEmbedding(features, wires=range(4))
        
        # 应用强纠缠层
        qml.templates.StronglyEntanglingLayers(weights, wires=range(4))
        
        # 返回所有量子比特的PauliZ期望值
        return [qml.expval(qml.PauliZ(i)) for i in range(4)]
    
    # 创建随机特征和权重
    features = np.random.rand(4) * np.pi
    weight_shape = qml.templates.StronglyEntanglingLayers.shape(n_layers=2, n_wires=4)
    weights = np.random.rand(*weight_shape) * np.pi
    
    # 执行电路
    results = template_circuit(features, weights)
    
    # 任务5: 绘制电路图并解释结构
    print("模板电路结构:")
    print(qml.draw(template_circuit)(features, weights))
    
    print("\n电路输出 (PauliZ期望值):")
    for i, result in enumerate(results):
        print(f"量子比特{i}: {result:.6f}")
    
    print("\n电路结构解释:")
    print("1. AngleEmbedding: 将4个经典特征值编码为量子态的相位")
    print("2. StronglyEntanglingLayers: 应用2层强纠缠层，每层包含:")
    print("   - 单量子比特旋转门 (RX, RY, RZ)")
    print("   - 纠缠门 (CNOT)")
    print("   - 这种结构对变分量子算法非常有用")
    
    return results

results4 = exercise4_solution()


练习5: 贝尔不等式测试

In [ ]:
print("\n练习5: 贝尔不等式测试 - 解答")

def exercise5_solution():
    # 任务1: 创建设备
    dev = qml.device('default.qubit', wires=2)
    
    # 任务2-3: 准备Bell态并计算CHSH不等式
    @qml.qnode(dev)
    def bell_test():
        # 准备Bell态
        qml.Hadamard(wires=0)
        qml.CNOT(wires=[0, 1])
        
        # 计算相关性
        # A₁ = Z⊗I, B₁ = I⊗Z
        # A₂ = X⊗I, B₂ = I⊗X
        
        term1 = qml.expval(qml.PauliZ(0) @ qml.PauliZ(1))  # <Z⊗Z> = <A₁B₁>
        term2 = qml.expval(qml.PauliZ(0) @ qml.PauliX(1))  # <Z⊗X> = <A₁B₂>
        term3 = qml.expval(qml.PauliX(0) @ qml.PauliZ(1))  # <X⊗Z> = <A₂B₁>
        term4 = qml.expval(qml.PauliX(0) @ qml.PauliX(1))  # <X⊗X> = <A₂B₂>
        
        # CHSH = <A₁B₁> + <A₁B₂> + <A₂B₁> - <A₂B₂>
        chsh = term1 + term2 + term3 - term4
        
        return [term1, term2, term3, term4, chsh]
    
    results = bell_test()
    
    print("Bell态的CHSH测试结果:")
    print(f"<Z⊗Z> = {results[0]:.6f}")
    print(f"<Z⊗X> = {results[1]:.6f}")
    print(f"<X⊗Z> = {results[2]:.6f}")
    print(f"<X⊗X> = {results[3]:.6f}")
    print(f"CHSH值 = {results[4]:.6f}")
    
    # 任务4: 验证结果是否超过经典极限2
    if abs(results[4]) > 2:
        print("\n结果超过了经典极限2，证明了量子纠缠的非局域性")
    else:
        print("\n结果未超过经典极限2，可能是由于噪声或实现错误")
    
    return results

results5 = exercise5_solution()


练习6: 量子隐形传态

In [ ]:
print("\n练习6: 量子隐形传态 - 解答")

def exercise6_solution():
    # 任务1: 创建设备
    dev = qml.device('default.qubit', wires=3)
    
    # 任务2: 实现量子隐形传态协议
    @qml.qnode(dev)
    def teleport(theta):
        # 准备初始状态在量子比特0
        qml.RY(2*theta, wires=0)
        
        # 创建Bell对：量子比特1和2
        qml.Hadamard(wires=1)
        qml.CNOT(wires=[1, 2])
        
        # Bell测量：量子比特0和1
        qml.CNOT(wires=[0, 1])
        qml.Hadamard(wires=0)
        
        # 模拟测量和经典通信（在实际设备上这会更复杂）
        # 在这里我们直接模拟最终结果，无需显式测量
        # 在量子比特2上应用校正，基于"假设"的测量结果
        
        # 注意：实际上，我们需要量子电路的中期测量或条件操作
        # 这里为了简化，我们应用所有可能的校正组合
        # 对于每个量子态，只有正确的校正会产生效果
        
        # 如果测量结果是|00⟩，无需更正
        # 如果测量结果是|01⟩，应用X门
        qml.PauliX(wires=2)
        # 撤销X门（为了模拟）
        qml.PauliX(wires=2)
        
        # 如果测量结果是|10⟩，应用Z门
        qml.PauliZ(wires=2)
        # 撤销Z门（为了模拟）
        qml.PauliZ(wires=2)
        
        # 如果测量结果是|11⟩，应用X和Z门
        qml.PauliZ(wires=2)
        qml.PauliX(wires=2)
        # 撤销XZ门（为了模拟）
        qml.PauliX(wires=2)
        qml.PauliZ(wires=2)
        
        # 返回最终状态的期望值
        return [qml.expval(qml.PauliZ(2)), qml.expval(qml.PauliX(2))]
    
    # 选择初始状态参数
    initial_theta = np.pi/4
    
    # 计算理论上的初始状态期望值（用于比较）
    initial_z = np.cos(2*initial_theta)
    initial_x = np.sin(2*initial_theta)
    
    # 执行量子隐形传态
    results = teleport(initial_theta)
    
    print("量子隐形传态结果:")
    print(f"初始状态: |ψ⟩ = cos({initial_theta}) |0⟩ + sin({initial_theta}) |1⟩")
    print(f"理论期望值: <Z> = {initial_z:.6f}, <X> = {initial_x:.6f}")
    print(f"传送后期望值: <Z> = {results[0]:.6f}, <X> = {results[1]:.6f}")
    
    # 任务3: 验证最终状态
    print("\n注意: 此实现为简化版，在实际设备上需要中期测量和经典通信")
    print("在完全的实现中，结果应与初始状态一致")
    
    return results

results6 = exercise6_solution()


练习7: 量子相位估计

In [ ]:
print("\n练习7: 量子相位估计 - 解答")

def exercise7_solution():
    # 任务1: 创建设备（需要4个量子比特：3个用于估计，1个用于特征值）
    dev = qml.device('default.qubit', wires=4)
    
    # 任务2: 实现简化版量子相位估计
    @qml.qnode(dev)
    def phase_estimation(phase):
        # 准备特征向量（最后一个量子比特）
        qml.PauliX(wires=3)
        
        # 初始化控制寄存器（前3个量子比特）
        for i in range(3):
            qml.Hadamard(wires=i)
        
        # 应用受控旋转门
        # 对于每个控制位，应用对应的幂次相位旋转
        with qml.control(1):
            qml.PhaseShift(phase * 2 * np.pi, wires=3)
        
        with qml.control(1):
            qml.PhaseShift(phase * 2 * np.pi * 2, wires=3)
        
        with qml.control(1):
            qml.PhaseShift(phase * 2 * np.pi * 4, wires=3)
        
        # 应用逆量子傅里叶变换
        # 实现3量子比特QFT†
        qml.adjoint(qml.QFT)(wires=range(3))
        
        # 测量控制寄存器
        return [qml.expval(qml.PauliZ(i)) for i in range(3)]
    
    # 设置真实相位
    true_phase = 0.25  # φ = 1/4
    
    # 执行量子相位估计
    results = phase_estimation(true_phase)
    
    print("量子相位估计结果:")
    print(f"真实相位: φ = {true_phase}")
    print(f"量子比特测量结果: {results}")
    
    # 解释结果
    print("\n对于φ = 0.25 (1/4)，理想的测量结果应该对应二进制010（即十进制2）")
    print("在实际实现中，我们可以从测量结果计算估计相位")
    print("此示例为简化版，完整实现需要测量概率分布并解释结果")
    
    return results

results7 = exercise7_solution()

print("\n所有练习解答已完成。这些解答展示了PennyLane的基本功能和量子计算的核心概念。")
print("建议尝试修改这些代码，探索不同的参数和电路结构，以加深对量子计算的理解。")
print("下一步: 学习变分量子电路和量子机器学习技术。")

## 💡 挑战任务

尝试扩展上面的练习，探索以下内容：
1. 更改电路参数并观察结果的变化
2. 尝试实现不同的量子态或算法
3. 可视化更多量子测量的结果